<a href="https://colab.research.google.com/github/Tar-ive/txst_open_datathon_2025/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# get data from web scraping


In [ ]:
!pip install firecrawl-py

In [ ]:
# Install with pip install firecrawl-py
from firecrawl import FirecrawlApp
from google.colab import userdata

app = FirecrawlApp(api_key=userdata.get('FIRECRAWL_API_KEY'))

crawl_result = app.crawl_url('https://www.reslife.txst.edu/get-started/options.html', params={
'limit': 5,
'scrapeOptions': {
	'formats': [ 'json' ],
  }
})

In [ ]:
# Install with pip install firecrawl-py
from firecrawl import FirecrawlApp
from google.colab import userdata

app = FirecrawlApp(api_key=userdata.get('FIRECRAWL_API_KEY'))

crawl_result = app.crawl_url('https://www.parking.txst.edu/about/statistics.html', params={
'limit': 5,
'scrapeOptions': {
	'formats': [ 'json' ],
  }
})

In [ ]:
import json
import csv

def json_to_csv(json_filepath, csv_filepath):
    """
    Converts a JSON file to a CSV file.

    Args:
        json_filepath: Path to the JSON file.
        csv_filepath: Path to the output CSV file.
    """
    try:
        with open(json_filepath, 'r', encoding='utf-8') as json_file:  # Handle potential encoding issues
            data = json.load(json_file)

        # Handle cases where the JSON data is a list of dictionaries or a single dictionary
        if isinstance(data, list):
            if data and isinstance(data[0], dict): # Check if it's a list of dictionaries
              keys = data[0].keys() # Get keys from the first dictionary (assuming all have same keys)
              with open(csv_filepath, 'w', newline='', encoding='utf-8') as csv_file:
                  writer = csv.DictWriter(csv_file, fieldnames=keys)
                  writer.writeheader()
                  writer.writerows(data)
            else:
              print(f"JSON file '{json_filepath}' contains a list, but not a list of dictionaries.  Cannot convert.")

        elif isinstance(data, dict): # If it's a single dictionary
            keys = data.keys()
            with open(csv_filepath, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=keys)
                writer.writeheader()
                writer.writerow(data)

        else:
          print(f"JSON file '{json_filepath}' is not a dictionary or a list of dictionaries. Cannot convert.")

    except FileNotFoundError:
        print(f"Error: JSON file '{json_filepath}' not found.")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in '{json_filepath}'.")
    except Exception as e:
        print(f"An error occurred: {e}")




json_to_csv("/content/documents_1 (1).json", "green_lots.csv")
json_to_csv("/content/documents_1.json", "dorm.csv")  # Example for a second file

In [ ]:
import pandas as pd

In [ ]:
dorm = pd.read_csv('/content/dorm.csv')
green_lots = pd.read_csv('/content/green_lots.csv')

In [ ]:
!pip install googlemaps
!pip install pandas

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=be9f1b98a7c981c395763663d0ef458403bc98bffca279d8346fc86d02fcdd42
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps


In [ ]:
import googlemaps
import pandas as pd
import time

# Initialize Google Maps client
API_KEY = 'GOOGLE_API_KEY'
gmaps = googlemaps.Client(key=API_KEY)

# Your university name and address
UNIVERSITY = "Texas State University, San Marcos, TX"  # Adjust this to your university

# Sample dorm data (replace with your actual data)
dorm_data = {
    'dorm_name': ['Alamito', 'Balcones', 'Bexar', 'Blanco','Bobcat Village Apartments','Brogdon', 'Butler',
    'Chautauqua', 'Cibolo', 'College Inn','Cypress', 'Elena Zamora O\'Shea',
    'Falls', 'First Five Freedom', 'Gaillardia', 'Jackson', 'Lantana',
    'Laurel', 'Mesquite', 'Retama', 'San Jacinto', 'San Marcos',
    'Sayers',  'Sterry', 'Tower'],
}

dorm_df = pd.DataFrame(dorm_data)

# Function to geocode each dorm
def geocode_dorm(dorm_name):
    try:
        # Create search query
        search_query = f"{dorm_name} Hall {UNIVERSITY}"

        # Get geocoding result
        result = gmaps.geocode(search_query)

        if result:
            location = result[0]['geometry']['location']
            formatted_address = result[0]['formatted_address']
            place_id = result[0]['place_id']

            return {
                'dorm_name': dorm_name,
                'latitude': location['lat'],
                'longitude': location['lng'],
                'formatted_address': formatted_address,
                'place_id': place_id
            }
        else:
            return {
                'dorm_name': dorm_name,
                'latitude': None,
                'longitude': None,
                'formatted_address': None,
                'place_id': None
            }
    except Exception as e:
        print(f"Error geocoding {dorm_name}: {str(e)}")
        return None

# Process all dorms and store results in a list
results = []
for dorm in dorm_df['dorm_name']:
    print(f"Geocoding {dorm}...")
    result = geocode_dorm(dorm)
    if result:
        results.append(result)
    time.sleep(0.1)  # Be respectful of the API rate limits

# Create a DataFrame from the geocoding results
geocoded_df = pd.DataFrame(results)

# Merge the geocoded data with the original dorm data using an inner join
merged_df = pd.merge(dorm_df, geocoded_df, on='dorm_name', how='inner')

# Save the merged DataFrame to a CSV file
merged_df.to_csv('dorm.csv', index=False)
print("\nResults saved to dorm.csv")

# Display the merged DataFrame (optional)
print("\nMerged DataFrame:")
print(merged_df.to_string())

# Display the data saved to dorm.csv (optional - you can read it back)
saved_df = pd.read_csv('dorm.csv')
print("\nData saved to dorm.csv:")
print(saved_df.to_string())

In [ ]:
dorm

,dorm_name,capacity,latitude,longitude,formatted_address,place_id
0,Alamito,505,29.890707,-97.942142,"102 Russell Cir, San Marcos, TX 78666, USA",ChIJXdq6QwCpXIYR3Llz2hvdYSY
1,Balcones,0,29.888411,-97.938351,"601 University Dr, San Marcos, TX 78666, USA",ChIJ3yStNnCoXIYRkulAp54VF3E
2,Bexar,202,29.886377,-97.948098,"100 Llano Cir, San Marcos, TX 78666, USA",ChIJ7Xcbw2-oXIYR1YezPAGFypY
3,Blanco,594,29.887135,-97.951914,"701 Moore St, San Marcos, TX 78666, USA",ChIJf9dt43CoXIYR0Mc75Frp3BI
4,Bobcat Village Apartments,0,29.893137,-97.922257,"1301 Aquarena Springs Dr, San Marcos, TX 78666...",ChIJdfj5QEmoXIYRv3-CbegaNPE
5,Brogdon,115,29.888299,-97.939900,"Brogdon Hall, 607 N Edward Gary St, San Marcos...",ChIJf317q2moXIYRK7pBUqDQWcU
6,Butler,236,29.887039,-97.938969,"500 N Edward Gary St, San Marcos, TX 78666, USA",ChIJOWRJE8qpXIYRkK3CBpJY7IE
7,Chautauqua,306,29.890471,-97.946585,"302 Student Center Dr, San Marcos, TX 78666, USA",ChIJ_VMPEiKpXIYRtUs52qENkwk
8,Cibilo,501,29.890119,-97.942420,"102 Russell Cir, San Marcos, TX 78666, USA",ChIJrcEROQCpXIYR5-DxagFJLJ0
9,College Inn,280,29.889331,-97.946390,"College Inn, San Marcos, TX 78666, USA",ChIJ-xRyaG6oXIYRieyzW_66PtM


In [ ]:
green_lots

,Location,Latitude,Longitude,Spaces
0,R56-SHC Thorpe,29.890614,97.945516,10
1,301-Llano Drive,29.886277,-97.948777,95
2,202-College Inn,29.889331,-97.946390,30
3,115-Residential,29.886604,-97.945323,85
4,Edward Gary St Garage,29.885832,-97.939751,196
5,112-San Jacinto Hall,29.887076,-97.944402,60
6,105-Admissions,29.886425,-97.942881,35
7,204-Jackson,29.890612,-97.944559,46
8,304-Academy Garage,29.885917,-97.947639,462
9,303-Blanco Garage,29.887755,-97.953557,271


In [ ]:
students = pd.read_csv('/content/students.csv')

In [ ]:
students

,Campus,Class,Department,Gender,Housing,Level,Semester,Year
0,Round Rock only,Doctoral-Professional,Physical Therapy,Female,Off-Campus,Doctoral,Summer,2023.0
1,Round Rock only,Doctoral-Professional,Physical Therapy,Female,Off-Campus,Doctoral,Summer,2023.0
2,Round Rock only,Master's,Nursing,Female,Off-Campus,Masters,Summer,2023.0
3,Round Rock only,Post-baccalaureate,Nursing,Female,Off-Campus,Post-baccalaureate,Summer,2023.0
4,Round Rock only,Senior,Respiratory Care,Female,Off-Campus,Undergraduate,Summer,2023.0
...,...,...,...,...,...,...,...,...
133103,San Marcos only,Junior,Art & Design,Male,Off-Campus,Undergraduate,Spring,2023.0
133104,San Marcos only,Junior,Journalism & Mass Communication,Male,Off-Campus,Undergraduate,Spring,2023.0
133105,San Marcos only,Junior,"Theatre, Dance, and Film",Male,Off-Campus,Undergraduate,Spring,2023.0
133106,San Marcos only,Junior,Geography and Environmental Studies,Male,Off-Campus,Undergraduate,Spring,2023.0
